# Metricas

In [1]:
from langchain import hub
from langchain.schema import StrOutputParser, Document
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_ollama import OllamaLLM
from langchain.embeddings import HuggingFaceBgeEmbeddings
from concurrent.futures import ThreadPoolExecutor
from langchain.document_loaders.base import BaseLoader
from langchain.docstore.document import Document
from typing import List, Optional
from functools import lru_cache
from langchain.storage import InMemoryStore
import time
import re
import os

In [2]:
phi= "phi3:mini"
llama = "llama3.2:latest"
url_llm = "http://localhost:11434"

In [3]:
phi_llm = OllamaLLM(model=phi, base_url=url_llm, temperature=0)
llama_llm = OllamaLLM(model=llama, base_url=url_llm, temperature=0)

### MODELO sentence-transformers/paraphrase-MiniLM-L6-v2

In [4]:
model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
emb = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

C:\Users\emolt\AppData\Local\Temp\ipykernel_17148\4035595027.py:4: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)


### EXPERIMENTO 7

In [5]:
vectorstore_7_txt = FAISS.load_local("faiss_index_proyectos_7_txt", embeddings=emb, 
                                 allow_dangerous_deserialization=True)

In [14]:
query_1 = "What is the objective of the project with grant agreement 740934?"
query_2 = "What is the total cost of the project with the acronym HYPERGRYD?"
query_3 = "How much funding was allocated for the project titled Transforming Research through Innovative Practices for Linked interdisciplinary Exploration?"
query_4 = "Which organisation played the role of coordinator in the grant agreement 777998?"
query_5 = "What topic does the project with the acronym INTERRFACE belong to?"
query_6 = "What legal basis was the project titled European Joint Programme on Radioactive Waste Management framed within?"
query_7 = "What type of proposal was the grant agreement 814416?"
query_8 = "To which master call was the project with the acronym G9NIGHT submitted?"
query_9 = "To which sub call was the project titled Electron Nanocrystallography submitted?"
query_10 = "Provide the grant agreement of 1 project which objective is related to artificial intelligence."
query_11 = "Provide the acronym of 1 project which objective is related to robotics."
query_12 = "Provide the title of 1 project which objective is related to geolocation."
query_13 = "Provide the objective of 1 project related to digital twin."
query_14 = "Provide the objective of 3 different projects related to corrosion."
query_15 = "Provide the title of 3 different projects which objective is related to offshore structures."
query_16 = "Provide the acronym of 3 different projects which objective is related to materials engineering."
query_17 = "Provide the grant agreement of 3 different projects which objective is related to nanocomposites."
query_18 = "Provide the name of an organisation that has participated in projects which objective is related to artificial intelligence."
query_19 = "Provide the name of an organisation which activity type is PRC and that has participated in projects which objective is related to robotics."
query_20 = "Provide the PIC of an organisation that is a small or medium enterprise and has participated in projects which objective is related to geolocation."
query_21 = "Provide the name of an organisation that has played the role of coordinator in projects which objective is related to digital twin."
query_22 = "Provide the PIC of a Spanish organisation that has participated in projects which objective is related to corrosion."
query_23 = "Provide the name of an european organisation that has participated in projects which objective is related to offshore structures."
query_24 = "Provide the PIC of an european small or medium enterprise that has participated in projects which objective is related to materials engineering."
query_25 = "Provide the name of an european small or medium enterprise that has played the role of coordinator in projects which objective is related to nanocomposites."

In [15]:
answer_1 ="The objective of grant agreement number 740934 under Horizon 2020 was not provided in your context. Therefore, I cannot provide an answer to this question based on the given information. If you have access to more details about project 740934 or would like assistance with a different query related to Horizon 2020 projects, please let me know!"
answer_2 ="The total funded cost for each project is explicitly mentioned next to its description, which allows us to directly answer your question without additional research or inference. The grant agreement number and the corresponding costs are as follows: HYPERGRYD (101036656) had a total funded cost of 5,987,874.5 euros; MSCA-IF project with gran agreement number 747921 also had a total funded cost of 183,454.8 euros (noting that this is the same amount as HYPERGRYD's); and finally, for grant agreement number 798271 under MSCA-IF with subcall H2020-MSCA-IF-2017, it had a total funded cost of 183,454.8 euros as well."
answer_3 ="The project titled Transforming Research through Innovative Practices for Linked interdisciplinary Exploration had a total funded cost of 137591.1 euros under grant agreement number 707706."
answer_4 ="The European Commission (EC) acted as the project leader for Grant Agreement No. 777998 under Horizon 2020, while a consortium of partners from various countries took on coordinator roles to manage and execute the research activities effectively across borders."
answer_5 ="INTERRFACE is related to fostering dialogue with stakeholders for better forest protection within legal frameworks, as part of Horizon 2020 project INTERFOR. It aims at promoting comparability and synchronization of national procedures and harmonizing evaluation mechanisms through various methods including surveys and interviews (WP3)."
answer_6 ="The European Joint Programme on Radioactive Waste Management (EURAD) was framed within Directive 2011/70/Euratom. This directive concerns measures to ensure a high common level of network and information security across the EU, which also recognizes cybersecurity's relevance for European economy preparedness among main stakeholders in this field."
answer_7 ="The grant agreement 814416 was framed within the legal basis H2020-EU.3.5.SOCIETAL CHALLENGES - Climate action, Environment, Resource Efficiency and Raw Materials under master call SC5-2014-2015 in Horizon 2020 projects funded by the European Union."
answer_8 ="The project G9NIGHT was submitted to master call H2020-MSCA-IF-2020."
answer_9 ="The Electron Nanocrystallography project was submitted to subcall H2020-MSCA-IF-2020 within master call H2020-MSCA-IF-2018."
answer_10 ="The grant agreement corresponding to an AI project is numbered as 101025896, titled 'Artificial intelligence for sustainable energy systems'. The objective of this Horizon Europe-funded research initiative (Grant Agreement Number: 101025896) was to develop and deploy AI technologies that can optimize the operation of smart grids. This project aimed at enhancing renewable integration, demand response management, energy efficiency in buildings, as well as providing robust cybersecurity measures for grid operations. The total cost allocated for this grant agreement amounted to 191852.16 euros and the funded costs were exactly equal due to full utilization of available resources under Horizon Europe's framework."
answer_11 ="The acronym is BADGER."
answer_12 ="The project titled 'Spatial Justice, Territorial Inequalities & Local Cohesion' focuses on geolocation aspects. It aims to understand and analyze territorial inequalities using spatial justice as an operational framework within local contexts like cities and their regions. The research will integrate historical data with modern archaeological reports for comprehensive analysis, employing non-invasive prospection techniques such as UAV flights and geophysical surveys to detect sites in the study area of interest."
answer_13 ="The objective of project FACTLOG is to deploy a real-time processing layer that integrates digital twins driven by domain models (knowledge) with data-derived experience for monitoring complex systems in process industries. This system, aimed at several industry sectors, facilitates cognition and control through the interoperability of observations, knowledge, and experiences within a unified framework called Cognitive Factory."
answer_14 ="""1. The ADDPRIME project aims to create an innovative solution that significantly increases metal's resistance to corrosion by using encapsulated inhibitors released through specific triggers, potentially reducing the annual costs of dealing with corrosion-related issues in Europe alone.
2. U-CROSS seeks to develop and validate ultrasonic sensors for real-time detection of early localized corrosion stages as well as monitoring damage progression when paint degrades, which could improve the aircraft industry's ability to detect both initiation and propagation of corrosion.
3. The objective of gran agreement 864905 is not explicitly stated in terms related to a specific project within Horizon 2020; however, it implies funding for research into predictive models that forecast how damage from different types of corrosion progresses when paint degrades, which could aid early detection and intervention."
"""
answer_15 ="""1) Delivering sub-sea technologies for new services at sea - BG-06-2014. This project focuses on developing capabilities to execute unmanned underwater operations in the deep ocean while assessing environmental impacts, which includes offshore structures as part of their service requirements.
2) Wave and Tidal Energy Conversion (WETISEC): A European Perspective - FP7-316095_EN. The project aims to evaluate seabed response to extreme waves for optimizing operational strategies, which is directly related to the design of offshore structures in terms of their resilience and efficiency.
3) Ocean Wave Energy Conversion (OWEC): A European Perspective - FP7-286509_EN. This project focuses on wave energy conversion technologies, which includes optimizing operational strategies for maritime renewable energy enterprises that involve offshore structures designed to harness ocean waves effectively."
"""
answer_16 ="The acronyms for three different projects with objectives related to materials engineering are NaMeS, EMMC-CSA, and TheLink."
answer_17 ="""1. Grant Agreement No.: 892354, Objective: The grant agreement focuses on understanding nanoparticle organization and its impact on material characteristics by investigating particle-liquid crystal interactions at submicron scales through examining chemistry and nanoparticle geometry effects using liquid crystals.
2. Grant Agreement No.: 685648, Objective: The project aims to develop the pilot scale production system for new generations of nanocomposites with multiple functions in gas adsorption, personal care and food by focusing on fast manufacturing spherical particles within nanoporous organic aerogels.
3. Grant Agreement No.: 713640 (NanoTRAINforGrowth II), Objective: The NanoTRAINforGrowth II fellowship program is designed to support experienced researchers in the field of nanotechnologies for applications across various sectors including medicine, energy, ICT, food and environment."
"""
answer_18 ="The European Union's Horizon 2020 programme includes a project called AI4theSciences, which involves Paris Sciences et Lettres University. This university is known for its excellence in training new generations and pushing the boundaries of research across various fields including artificial intelligence (AI)."
answer_19 ="The European Robotics Research Infrastructures Network (TERRINet) is an organisation with activity type PRC that has participated in projects related to robotics, specifically focusing on building a world-class network for research infrastructure and boosting scientific research and technological innovation."
answer_20 ="The organisation GEO-MONTAN GEOLOGUS, KORNYEZETVEDOMEGUJULO ENERGETIKAI TOLMACSFORDITOKFT with PIC 951946264 is a small or medium enterprise that participated in Horizon 2020 projects related to geolocation."
answer_21 ="The IOTWINS (Internet-of-Things Twins) organisation has played a coordinating role in projects related to digital twin within Horizon 2020. They focus on developing hierarchical models for manufacturing plants and facility management, aiming at optimizing performance metrics through anomaly detection and diagnosis using IoT data streams."
answer_22 ="The project Development of an innovative, low-cost coating for steel structures against chloride attack (PIC code EVOCOAT) involved the Spanish company Aqualimia S.L., which is based in Barcelona and specializes in developing coatings to protect infrastructures from corrosion."
answer_23 ="The European Seakeeping Centre (ESC) was involved in Horizon 2020 project (SeaWiFy) with the aim of developing a low-cost, autonomous underwater vehicle for monitoring offshore structures. The ESC is an EU funded research centre based at DTU Aqua and has been actively participating in projects related to marine technology since its establishment."
answer_24 ="I'm sorry, but I cannot provide the specific PIC of an enterprise without access to current databases or project lists. However, based on the context provided about Horizon 2020 projects related to materials engineering and extrusion processes in Europe, a hypothetical example could be 'Materials Innovation for Sustainable Extrusion' by EuroMix GmbH from Germany that participated in such initiatives."
answer_25 ="The name of an European small or medium enterprise that has played the role of coordinator in projects related to nanocomposites under Horizon 2020 is ZENIT. They have been involved with CO-PILOT and other initiatives aimed at enhancing innovation management capacities for SMEs, particularly focusing on nanotechnology applications like nanocomposites."

In [16]:
answer_1_correct="The project’s objective is to combat violent extremism by analyzing its root causes, developing preventive and repressive measures, and countering extremist narratives through collaboration with civil society and LEAs, all while upholding fundamental rights."
answer_2_correct ="The total cost of the project with the acronym HYPERGRYD (grant agreement 101036656) was €5,987,875.00."
answer_3_correct ="Transforming Research through Innovative Practices for Linked interdisciplinary Exploration” (TRIPLE), identified by grant agreement 863420, received a total EU contribution of € 5,626,548.75. This funding was allocated as part of Horizon 2020 under the “EXCELLENT SCIENCE – Research Infrastructures"
answer_4_correct ="""the role of coordinator in the grant agreement 777998. The participation cost of the organisation with PIC 960782479 in the grant agreement 777998.
The organisation with Participant Identification Code (PIC) 960782479 participated in the grant agreement 777998. The name of this organisation is UNIVERSIDADE NOVA DE LISBOA. The organisation with PIC 960782479 is not a small or medium-sized enterprise. The organisation with PIC 960782479 develops an activity of type HES. The organisation with PIC 960782479 is based in the country PT, codified under ISO 3166. The organisation with PIC 960782479 played the role of coordinator in the grant agreement 777998. The participation cost of the organisation with PIC 960782479 in the grant agreement 777998 was 409500.0 euros. The total amount funded to the organization with PIC 960782479 in the grant agreement 777998 was 409500.0 euros.
"""
answer_5_correct ="The grant agreement 824330 was framed within the topic LC-SC3-ES-5-2018-2020TSO – DSO – Consumer: Large-scale demonstrations of innovative grid services through demand response, storage and small-scale (RES) generation. The grant agreement 824330 was framed within the master call H2020-LC-SC3-2018-2019-2020. The grant agreement 824330 was framed within the subcall H2020-LC-SC3-2018-ES-SCC."
answer_6_correct ="The grant agreement 847593 was framed within the legal basis H2020-EuratomEuratom."
answer_7_correct ="The grant agreement 814416 was a Research and Innovation Action (RIA) proposal."
answer_8_correct ="The grant agreement 101036041 was framed within the master call H2020-MSCA-NIGHT-2020bis."
answer_9_correct ="The grant agreement 956099 was framed within the legal basis H2020-EU.1.3.EXCELLENT SCIENCE - Marie Skłodowska-Curie Actions."

In [21]:
import langchain

num_queries = 9
results = {}  # Almacena respuestas completas del LLM
scores = {}   # Almacena puntuaciones extraídas
rows = []     # Para guardar resultados estructurados

for i in range(1, num_queries + 1):
    question = eval(f"query_{i}")
    answer = eval(f"answer_{i}")
    answer_correct = eval(f"answer_{i}_correct")
    print()
    prompt = f"""
    You are a rigorous evaluator. You will compare a system-generated answer to the known correct answer and evaluate its accuracy.

    INSTRUCTIONS:
    1. Analyze the question, the correct answer, and the generated answer.
    2. Evaluate EXCLUSIVELY the match between the generated answer and the correct answer.
    3. Assigns a score from 1 to 5 based on this scale:

    1 = Completely incorrect (no match in any respect)
    2 = Mostly incorrect (minimal match)
    3 = Partially correct (some elements match)
    4 = Mostly correct (only minor errors)
    5 = Completely correct (exact match or equivalent).

    DATA:
    Question: {question}
    Correct answer: {answer_correct}
    Response generated: {answer}

    STRICT OUTPUT FORMAT (no deviations):
    Justification: [1-2 sentence concise explanation of differences/similarities]
    Score: [Integer from 1 to 5]
    """



    response = llama_llm.invoke(prompt)  
    results[i] = response
    
    print(f"Explicación del evaluador:\n{results[i]}")
    print("\n---\n")


Explicación del evaluador:
Justification:
The generated answer is completely incorrect as it does not provide any information about the objective of grant agreement 740934, whereas the correct answer provides a detailed description of the project's goal.

Score: 1

---


Explicación del evaluador:
Justification:
The generated answer partially matches the correct answer, but it includes additional information about other grant agreements and their costs. The main difference is that the correct answer only mentions the HYPERGRYD project, while the generated answer also mentions two other projects.

Score: 4

---


Explicación del evaluador:
Justification: The generated answer only provided the total funded cost and grant agreement number, whereas the correct answer included additional information about the EU contribution and program name. Additionally, the correct answer mentioned that this funding was allocated as part of Horizon 2020.

Score: 4

---


Explicación del evaluador:
Justi